# <img align="left" src="assets/panda.jpeg" width="100" height="100" style="border-radius: 50%; margin-right: 20px">  
# Carteira de pedidos

> A principal proposta desse projeto é automatizar o tratamento desse relatório para atualizar um dashboard para análise de pedidos de compras dos CDs e Lojas

### Tratamento para a Base Relatório

In [239]:
# bibliotecas usadas no projeto
from datetime import datetime, timedelta
import pandas as pd
import locale
import csv
import os

In [240]:
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [241]:
# caminho dos arquivos txt
arquivos_flex = "Relatorios_pedidos/"

In [242]:
# número de colunas dos arquivos
total_colunas = 31

### Base de seis relatórios do ERP extraídos no formato txt - r1 ao r6

In [243]:
'''O código abaixo percorre por todos os arquivos no diretório Relatorios_pedidos,
primeiro processa o arquivo r1.txt e os demais posteriormente,
validando a definição do número de colunas.'''

r1_file = os.path.join(arquivos_flex, "r1.txt")
if os.path.exists(r1_file):
    with open(r1_file, "r", encoding="iso-8859-1") as infile, open("temp.txt", "w", encoding="iso-8859-1", newline="") as outfile:
        reader = csv.reader(infile, delimiter="|")
        writer = csv.writer(outfile, delimiter="|")

        for row in reader:
            if len(row) == total_colunas:
                writer.writerow(row)

# carrega os outros arquivos no diretório
for filename in os.listdir(arquivos_flex):
    if filename.endswith(".txt") and filename != "r1.txt":
        with open(os.path.join(arquivos_flex, filename), "r", encoding="iso-8859-1") as infile, open("temp.txt", "a", encoding="iso-8859-1", newline="") as outfile:
            reader = csv.reader(infile, delimiter="|")
            writer = csv.writer(outfile, delimiter="|")

            for row in reader:
                if len(row) == total_colunas:
                    writer.writerow(row)

In [244]:
# leitura e atribuição do arquivo temp para o objeto "base"
base = pd.read_csv("temp.txt", sep="|", encoding="iso-8859-1")

In [245]:
# ppós isso removi o arquivo temp 
os.remove("temp.txt")

In [246]:
# status da base original antes de qualquer tratamento
print("A Base original atualmente possui {} linhas e {} colunas".format(base.shape[0], base.shape[1]))

A Base original atualmente possui 1523283 linhas e 31 colunas


### Relatório de pedidos com status cancelados

In [247]:
# leitura da base de pedidos cancelados
base_c = pd.read_csv("Relatorio_cancelados/cancelados.txt", sep="|", encoding="iso-8859-1", low_memory=False)

In [248]:
# merge da "base" com a "base_cancelados"
base = base.merge(base_c[["Número", "St"]], left_on="Num Pedido", right_on="Número", how="left")

### Relatório de pedidos com status pendente

In [249]:
# leitura da base de pedidos pendentes
base_p = pd.read_csv("Relatorio_pendentes/pendentes.txt", sep="|", encoding="iso-8859-1", low_memory=False)

In [250]:
# merge da "base" com a "base_pendentes"
base = base.merge(base_p[["Número", "Autorização"]], left_on="Num Pedido", right_on="Número", how="left")

In [251]:
# quantidade de linhas com pedidos cancelados: "True"
base_linhas_st = (base["St"] == "C").value_counts()
base_linhas_st

St
False    1323156
True      200127
Name: count, dtype: int64

In [252]:
# removi da base todos os pedidos com status "C" de cancelado
base_relatorio = base.drop(base[base["St"] == "C"].index)

In [253]:
# status da base
print("Base após remover pedidos cancelados: " + str(base_relatorio.shape))

Base após remover pedidos cancelados: (1323156, 35)


In [254]:
# quantidade de linhas com pedidos pendentes: 'True'
base_linhas_pend = (base["Autorização"] == "P").value_counts()
base_linhas_pend

Autorização
False    1521078
True        2205
Name: count, dtype: int64

In [255]:
# removi da base todos os pedidos com status "P" de pendentes
base_relatorio = base_relatorio.drop(base_relatorio[base_relatorio["Autorização"] == "P"].index)

In [256]:
# status da base
print("Base após remover pedidos pendentes: " + str(base_relatorio.shape))

Base após remover pedidos pendentes: (1320951, 35)


In [257]:
# exclusão das linhas excedentes dos merges
base_relatorio = base_relatorio.drop("Unnamed: 30", axis=1)
base_relatorio = base_relatorio.drop("Número_x", axis=1)
base_relatorio = base_relatorio.drop("St", axis=1)
base_relatorio = base_relatorio.drop("Número_y", axis=1)
base_relatorio = base_relatorio.drop("Autorização", axis=1)

In [258]:
# status da base
print("Base após remover colunas excedentes: " + str(base_relatorio.shape))

Base após remover colunas excedentes: (1320951, 30)


In [259]:
# filtro por datas vazias
base_relatorio = base_relatorio[pd.isna(base_relatorio["Data NFE"])]

In [260]:
# número de linhas vazias na coluna Origem
numero_de_linhas_com_0 = (base_relatorio["Origem"] == 0).sum()
print(numero_de_linhas_com_0)

9


In [261]:
# copia o número do pedido quando o valor da coluna "Origem" for igual a 0.
base_relatorio.loc[base_relatorio["Origem"] == 0, "Origem"] = base_relatorio["Num Pedido"]

In [262]:
# número de linhas vazias após tratamento deve ser 0
numero_de_linhas_com_0 = (base_relatorio["Origem"] == 0).sum()
print(numero_de_linhas_com_0)

0


In [263]:
# conversão das colunas para objetos de Data.
base_relatorio["Data"] = pd.to_datetime(base_relatorio["Data"], format="%d/%m/%Y", errors="coerce")
base_relatorio["Previsao"] = pd.to_datetime(base_relatorio["Previsao"], format="%d/%m/%Y", errors="coerce")
base_relatorio["Data Baixa"] = pd.to_datetime(base_relatorio["Data Baixa"], format="%d/%m/%Y", errors="coerce")

In [264]:
# consulta de pedidos por forma de aquisicao
base_linhas_FA = (base_relatorio["Forma Aquisicao"]).value_counts()
base_linhas_FA

Forma Aquisicao
CC    44315
SI    27924
DT    24443
CD    23581
Name: count, dtype: int64

In [265]:
# removendo os dados da coluna Forma Aquisicao.
base_relatorio["Forma Aquisicao"] = ''

In [266]:
# conversão da coluna ao tipo float.
base_relatorio["Valor"] = base_relatorio["Valor"].str.replace(".", "").str.replace(",", ".").astype(float)

In [267]:
# ordena a coluna "Valor" do maior para o menor valor
base_relatorio = base_relatorio.sort_values(by="Valor", ascending=False)

In [268]:
# status da base
print("A base_relatorio atualmente possui {} linhas e {} colunas".format(base_relatorio.shape[0], base_relatorio.shape[1]))

A base_relatorio atualmente possui 120263 linhas e 30 colunas


In [269]:
# função para criar a coluna "Chave" no dataframe com base na validação abaixo
def criar_chave(row):
    if pd.notna(row["Origem"]):
        origem_str = str(int(row["Origem"]))  # conversões para string
        cod_produto_str = str(row["Cod. Produto"])
        return origem_str + cod_produto_str
    else:
        num_pedido_str = str(row["Num Pedido"])
        cod_produto_str = str(row["Cod. Produto"])
        return num_pedido_str + cod_produto_str

In [270]:
# cria a nova coluna "Chave" aplicando a função a cada linha do dataframe
base_relatorio["Chave"] = base_relatorio.apply(criar_chave, axis=1)

In [271]:
# reordenação de colunas
colunas = base_relatorio.columns.tolist()
colunas = ["Chave"] + [coluna for coluna in colunas if coluna != "Chave"]
base_relatorio = base_relatorio[colunas]

In [272]:
# base pendentes.txt
base_p = pd.read_csv("Relatorio_pendentes/pendentes.txt", sep="|", encoding="iso8859-1", low_memory=False)

In [273]:
# merge com a coluna "Dcto" da base pendentes.txt
base_relatorio = base_relatorio.merge(base_p[["Número", "Dcto"]], left_on="Num Pedido", right_on="Número", how="left")

In [274]:
# merge com a coluna "Nome Usuário" da base pendentes.txt
base_relatorio = base_relatorio.merge(base_p[["Número", "Nome Usuário"]], left_on="Num Pedido", right_on="Número", how="left")

In [275]:
# reordenação de colunas
colunas = list(base_relatorio.columns)
colunas.remove("Dcto")
colunas.insert(colunas.index("Chave") + 1, "Dcto")
base_relatorio = base_relatorio[colunas]

In [276]:
# função para determinar o valor da coluna "Tipo Pedido" com base na validação
def determinar_tipo_pedido(row):
    if "7078" in str(row["Dcto"]) or "7069" in str(row["Dcto"]) or "7709" in str(row["Dcto"]):
        return "Automatico"
    else:
        return "Manual"

In [277]:
# cria a nova coluna "Tipo Pedido" aplicando a função "determinar_tipo_pedido" do dataframe
base_relatorio["Tipo Pedido"] = base_relatorio.apply(determinar_tipo_pedido, axis=1)

In [278]:
# reordenação de colunas
colunas = list(base_relatorio.columns)
colunas.remove("Tipo Pedido")
colunas.insert(colunas.index("Dcto") + 1, "Tipo Pedido")
base_relatorio = base_relatorio[colunas]

In [279]:
# converte as colunas para string e tratar NaNs
base_relatorio['Num Pedido'] = base_relatorio['Num Pedido'].astype(str)
base_relatorio['Origem'] = base_relatorio['Origem'].fillna('nan').apply(lambda x: str(int(x)) if x != 'nan' else 'nan')

In [282]:
# identificar os números de pedidos que aparecem em ambas as colunas
pedidos_repetidos = set(base_relatorio['Num Pedido']).intersection(set(base_relatorio['Origem']))

In [283]:
# função para determinar o valor da coluna "Classificacao Pedido" com base na validação
def classificar_pedidos(row):
    num_pedido_str = str(row['Num Pedido']) # definição para a função que classifica cada linha
    origem_str = 'nan' if pd.isna(row['Origem']) else str(row['Origem'])

    if num_pedido_str in pedidos_repetidos:
        return 'Saldo'
    elif origem_str == 'nan':
        return 'Original'
    else:
        return 'Saldo'

In [284]:
# cria a nova coluna "Classificacao Pedido" aplicando a função a cada linha do dataframe
base_relatorio["Classificacao Pedido"] = base_relatorio.apply(classificar_pedidos, axis=1)

In [285]:
# reordenação de colunas
colunas = list(base_relatorio.columns)
colunas.remove("Classificacao Pedido")
colunas.insert(colunas.index("Tipo Pedido") + 1, "Classificacao Pedido")
base_relatorio = base_relatorio[colunas]

In [286]:
# reordenação de colunas
colunas = list(base_relatorio.columns)
colunas.remove("Nome Usuário")
colunas.insert(colunas.index("Classificacao Pedido") + 1, "Nome Usuário")
base_relatorio = base_relatorio[colunas]

In [287]:
# exclusão de colunas excedentes
base_relatorio = base_relatorio.drop("Número_x", axis=1)
base_relatorio = base_relatorio.drop("Número_y", axis=1)

In [288]:
# colunas para remover duplicatas
colunas_chave = ['Nome Usuário', 'Transacao', 'Origem', 'Fornecedor', 'Nome do Fornecedor', 'Data',
                 'Previsao', 'Data Baixa', 'Data Criação Agenda', 'Status Agenda', 'Data Agenda',
                 'Data NFE', 'Cod. Produto', 'Nome do Produto', 'Marca', 'Num Pedido', 'Unid', 'Qtde',
                 'Qtde. Emb', 'Valor', 'Distribuicao', 'Forma Aquisicao', 'Departamento', 'Grupo',
                 'Cod. Barras', 'NF-e Chave de Acesso', 'NF-e Transacao', 'NF-e Qtde', 'NF-e Valor',
                 'Nº Nota Fiscal', 'Tipo Frete']

In [289]:
# status da base 
base_relatorio.shape

(120263, 35)

In [290]:
# remove as duplicatas com base nas colunas especificadas na variável colunas_chave
base_relatorio = base_relatorio.drop_duplicates(subset=colunas_chave)

In [291]:
# status da base após remoção de dados duplicados
base_relatorio.shape

(73888, 35)

In [292]:
# remove as duplicatas com base apenas na coluna "Chave"
base_relatorio = base_relatorio.drop_duplicates(subset="Chave")

In [293]:
# status da base após remoção de todos os dados duplicados
base_relatorio.shape

(56370, 35)

In [294]:
# exclui as duas colunas abaixo
base_relatorio = base_relatorio.drop("Chave", axis=1)
base_relatorio = base_relatorio.drop("Dcto", axis=1)

In [295]:
# fim do tratamento base_relatorio
print("A base_relatorio possui {} linhas e {} colunas".format(base_relatorio.shape[0], base_relatorio.shape[1]))

A base_relatorio possui 56370 linhas e 33 colunas


In [296]:
# obtém a data atual e inicia tratamento para base_dashboard
data_atual = datetime.now()

In [297]:
# calcula a data com até dois meses anteriores ao mês atual
data_limite_2_meses = data_atual - timedelta(days=data_atual.day + 1)
data_limite_2_meses = data_limite_2_meses.replace(day=1)

In [298]:
# calcula a data com 7 dias inferior ao dia atual
data_limite_7_dias = data_atual - timedelta(days=8)

In [299]:
''' O código abaixo fará o filtro, excluindo da base as linhas com dados que possui
a data de previsao inferior a 7 dias '''

base_pedidos = base_relatorio[(base_relatorio['Previsao'] >= data_limite_2_meses) & (base_relatorio['Previsao'] >= data_limite_7_dias)]


In [300]:
# reatribuição entre dataframes
base_relatorio = base_pedidos

In [301]:
# status da base
base_relatorio.shape

(56067, 33)

### Tratamento final para dar carga no Dashboard da carteira de pedidos

In [302]:
# exibe todas as colunas e linhas do DF
#pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [303]:
# base cadastro SKUs
base_cadastro = pd.read_excel('Base_cadastro/base_cadastro.xlsx')

In [304]:
# cria a coluna Pedido
###base_relatorio['Pedido'] = base_relatorio.apply(lambda row: row['Num Pedido'] if pd.isna(row['Origem']) else row['Origem'], axis=1)
base_relatorio['Pedido'] = base_relatorio.apply(lambda row: row['Num Pedido'] if pd.isna(row['Origem']) or row['Origem'] in ['', 'nan'] else row['Origem'], axis=1)

In [305]:
# faz o merge com a base_cadastro
base_relatorio = base_relatorio.merge(base_cadastro[["ProdutoCodigo", "Categoria"]], right_on="ProdutoCodigo", left_on="Cod. Produto", how="left")

In [306]:
# cria a coluna Local_ent
base_relatorio.loc[:, 'Local_ent'] = base_relatorio['Unid'].apply(lambda x: 'CD' if x in [3, 61, 745] else 'Loja')
#base_relatorio.loc[:, 'Local_ent'] = base_relatorio['Unid'].apply(lambda x: 'CD' if x in [3, 61, 745] else 'Loja').copy()

In [307]:
base_relatorio = base_relatorio.drop("Transacao", axis=1)
base_relatorio = base_relatorio.drop("Origem", axis=1)
base_relatorio = base_relatorio.drop("ProdutoCodigo", axis=1)
base_relatorio = base_relatorio.drop("Marca", axis=1)

In [308]:
# cria coluna grupo certo
base_relatorio['Grupo_certo'] = base_relatorio.apply(lambda row: row['Categoria'].capitalize() if pd.notna(row['Categoria']) and row['Grupo'] != row['Categoria'] else row['Grupo'].capitalize(), axis=1)

In [309]:
# removendo colunas excedentes
base_relatorio = base_relatorio.drop("Data Criação Agenda", axis=1)
base_relatorio = base_relatorio.drop("Status Agenda", axis=1)
base_relatorio = base_relatorio.drop("Data Agenda", axis=1)
base_relatorio = base_relatorio.drop("Data NFE", axis=1)
base_relatorio = base_relatorio.drop("Num Pedido", axis=1)
base_relatorio = base_relatorio.drop("Distribuicao", axis=1)
base_relatorio = base_relatorio.drop("Forma Aquisicao", axis=1)
base_relatorio = base_relatorio.drop("Categoria", axis=1)
base_relatorio = base_relatorio.drop("Grupo", axis=1)
base_relatorio = base_relatorio.drop("Cod. Barras", axis=1)
base_relatorio = base_relatorio.drop("NF-e Chave de Acesso", axis=1)
base_relatorio = base_relatorio.drop("NF-e Transacao", axis=1)
base_relatorio = base_relatorio.drop("NF-e Qtde", axis=1)
base_relatorio = base_relatorio.drop("NF-e Valor", axis=1)
base_relatorio = base_relatorio.drop("Nº Nota Fiscal", axis=1)
base_relatorio = base_relatorio.drop("Tipo Frete", axis=1)

In [310]:
colunas = ['Pedido', 'Tipo Pedido', 'Classificacao Pedido', 'Local_ent', 'Unid', 'Data', 'Previsao', 
           'Data Baixa', 'Nome Usuário', 'Fornecedor', 'Nome do Fornecedor', 'Cod. Produto', 'Nome do Produto', 
           'Departamento', 'Grupo_certo', 'Qtde. Emb', 'Qtde', 'Valor'] + [coluna for coluna in base_relatorio.columns if coluna not in ['Pedido', 'Tipo Pedido', 'Classificacao Pedido', 'Local_ent', 'Unid', 'Data', 'Previsao', 'Data Baixa', 'Nome Usuário', 'Fornecedor', 'Nome do Fornecedor', 
           'Cod. Produto', 'Nome do Produto', 'Departamento', 'Grupo_certo', 'Qtde. Emb', 'Qtde', 'Valor']]

base_relatorio = base_relatorio[colunas]

In [311]:
renomear_colunas = {
    'Pedido': 'PEDIDO',
    'Classificacao Pedido': 'Classificao do pedido',
    'Local_ent': 'LOCAL_ENT',
    'Unid': 'N_LJ',
    'Data': 'DT_EMISSAO',
    'Previsao': 'DT_ENTREGA',
    'Data Baixa': 'DT_BAIXA',
    'Nome Usuário': 'NOME USUARIO',
    'Fornecedor': 'COD_FORN',
    'Nome do Fornecedor': 'DESC_FORN',
    'Cod. Produto': 'COD_SKU',
    'Nome do Produto': 'DESC_SKU',
    'Departamento': 'DEP',
    'Grupo_certo': 'GRUPO',
    'Qtde. Emb': 'CX',
    'Qtde': 'PED_UNIDS',
    'Valor': 'PED_$',
}
base_relatorio.rename(columns=renomear_colunas, inplace=True)

In [312]:
# bases adicionais
base_semana = pd.read_excel('Bases_externas/base_semana_anual.xlsx')
base_rup = pd.read_excel('Bases_externas/rup.xls')
base_agenda = pd.read_excel('Bases_externas/agenda.xls')

In [313]:
# converte coluna 'Data' para o tipo datetime em base_semana
base_semana['Data'] = pd.to_datetime(base_semana['Data'], format='%d/%m/%Y')

In [314]:
# merge entre base_relatorio e base_semana
base_dashboard = pd.merge(base_relatorio, base_semana[['Data', 'Semana', 'SEM']], left_on='DT_ENTREGA', right_on='Data', how='left')

In [315]:
# merge entre base_dashboard e base_rup
base_dashboard = pd.merge(base_dashboard, base_rup[['Prod Código', 'Ruptura']], left_on='COD_SKU', right_on='Prod Código', how='left')

In [316]:
# add o '-' onde estiver NaN na coluna "QT_LJS_RUP"
base_dashboard['QT_LJS_RUP'] = base_dashboard['Ruptura'].fillna('-')

In [317]:
base_dashboard[['Prod Código', 'Ruptura']] = base_dashboard[['Prod Código', 'Ruptura']].fillna('-')

In [318]:
# renomeia a coluna Pedido da base_agenda
renomear_colunas = { 'PEDIDO': 'Pedido' }
base_agenda.rename(columns=renomear_colunas, inplace=True)

In [319]:
base_agenda = base_agenda.drop_duplicates(subset='Pedido') # removendo dados duplicados 

In [320]:
base_agenda = base_agenda.sort_values(by='AGENDAMENTO', ascending=False)

In [321]:
path = "Base_relatorio/agenda.xlsx"

In [322]:
base_agenda.to_excel(path, index=False)

In [323]:
# converte a coluna PEDIDO para string e remove o ponto flutuante
base_dashboard['PEDIDO'] = base_dashboard['PEDIDO'].apply(lambda x: str(x).split('.')[0] if pd.notnull(x) else x)

In [324]:
# converte a coluna Pedido de base_agenda para string
#base_agenda['Pedido'] = base_agenda['Pedido'].astype(str)
base_agenda['Pedido'] = base_agenda['Pedido'].apply(lambda x: str(int(x)) if not pd.isna(x) else 'nan')

In [325]:
# merge das bases usando a coluna PEDIDO/Pedido
base_dashboard = pd.merge(base_dashboard, base_agenda[['Pedido', 'AGENDAMENTO']], left_on='PEDIDO', right_on='Pedido', how='left')

In [326]:
# criando a coluna DT_AGENDA com base em AGENDAMENTO e DT_ENTREGA
base_dashboard['DT_AGENDA'] = base_dashboard['AGENDAMENTO']
# para linhas sem AGENDAMENTO, usar o mês de DT_ENTREGA
base_dashboard.loc[base_dashboard['DT_AGENDA'].isna(), 'DT_AGENDA'] = base_dashboard['DT_ENTREGA'].dt.strftime('SEM AGENDA %b').str.upper()

In [327]:
# exclusão de colunas excedentes
base_dashboard = base_dashboard.drop("Data", axis=1)
base_dashboard = base_dashboard.drop("Semana", axis=1)
base_dashboard = base_dashboard.drop("Prod Código", axis=1)
base_dashboard = base_dashboard.drop("Ruptura", axis=1)
base_dashboard = base_dashboard.drop("Pedido", axis=1)
base_dashboard = base_dashboard.drop("AGENDAMENTO", axis=1)

In [328]:
# ordenação final
colunas = ['PEDIDO', 'Tipo Pedido', 'Classificao do pedido', 'LOCAL_ENT', 'N_LJ', 'SEM', 'DT_EMISSAO',
'DT_ENTREGA', 'DT_BAIXA', 'NOME USUARIO', 'COD_FORN', 'DESC_FORN','COD_SKU','DESC_SKU','DEP', 'GRUPO', 
'CX', 'QT_LJS_RUP', 'PED_UNIDS', 'PED_$', 'DT_AGENDA'] + [coluna for coluna in base_dashboard.columns if coluna not in ['PEDIDO', 
'Tipo Pedido', 'Classificao do pedido', 'LOCAL_ENT', 'N_LJ', 'SEM', 'DT_EMISSAO', 'DT_ENTREGA', 'DT_BAIXA', 'NOME USUARIO', 
'COD_FORN', 'DESC_FORN','COD_SKU','DESC_SKU', 'DEP', 'GRUPO', 'CX', 'QT_LJS_RUP', 'PED_UNIDS', 'PED_$', 'DT_AGENDA']]

base_dashboard = base_dashboard[colunas]

In [329]:
base_dashboard = base_dashboard.sort_values(by='PEDIDO') # ordena do menor para o maior na coluna "PEDIDO"

In [330]:
# Função para conversão das colunas em númerico
def convert_with_comma_to_number(val):
    if isinstance(val, str):
        val = val.replace(',', '').strip()
    return pd.to_numeric(val, errors='coerce')

# aplicando a função de conversão nas colunas abaixo
base_dashboard['PEDIDO'] = base_dashboard['PEDIDO'].apply(convert_with_comma_to_number)
base_dashboard['CX'] = base_dashboard['CX'].apply(convert_with_comma_to_number)
base_dashboard['PED_UNIDS'] = base_dashboard['PED_UNIDS'].apply(convert_with_comma_to_number)
base_dashboard['PED_$'] = base_dashboard['PED_$'].apply(convert_with_comma_to_number)

In [331]:
# status da base após etapa final em linhas x colunas
base_dashboard.shape

(56067, 21)

In [332]:
# caminho para salvar a base
caminho_arquivo_xlsx = "Base_relatorio/base_dashboard.xlsx"

In [333]:
# salvando base tratada em excel
base_dashboard.to_excel(caminho_arquivo_xlsx, index=False)